## Teams

Teams define how groups of agents communicate to address tasks. AgentChat provides several preset team configurations to simplify building multi-agent applications.

```{include} ../warning.md

```

A team may consist of a single agent or multiple agents. An important configuration for each team involves defining the order in which agents send messages and determining when the team should terminate.

In the following section, we will begin by defining agents.

In [1]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.task import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

# Set up a log handler to print logs to the console.
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)


async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."


get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

tool_use_agent = ToolUseAssistantAgent(
    "tool_use_agent",
    system_message="You are a helpful assistant that solves tasks by only using your tools.",
    model_client=model_client,
    registered_tools=[get_weather_tool],
)

### RoundRobinGroupChat

A team where agents take turns sending messages (in a round robin fashion) until a termination condition is met. 

In [2]:
round_robin_team = RoundRobinGroupChat([tool_use_agent, writing_assistant_agent])
round_robin_team_result = await round_robin_team.run(
    "Write a Haiku about the weather in Paris", termination_condition=MaxMessageTermination(max_messages=1)
)


--------------------------------------------------------------------------- 
[2024-10-20T09:01:04.692283]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-10-20T09:01:05.961670], tool_use_agent:

Golden sun above,  
Paris basks in warmth and light,  
Seine flows in sunshine.
--------------------------------------------------------------------------- 
[2024-10-20T09:01:05.962309], Termination:

Maximal number of messages 1 reached, current message count: 1

Similarly, we can define a team where the agents solve a problem by _writing and executing code_ in a round-robin fashion. 

```python 
async with DockerCommandLineCodeExecutor(work_dir="coding") as code_executor:
    code_executor_agent = CodeExecutorAgent(
        "code_executor", code_executor=code_executor)
    code_execution_team = RoundRobinGroupChat([writing_assistant_agent, code_executor_agent])
    code_execution_team_result = await code_execution_team.run("Create a plot of NVDIA and TSLA stock returns YTD from 2024-01-01 and save it to 'nvidia_tesla_2024_ytd.png", termination_condition=MaxMessageTermination(max_messages=12))
```

### SelectorGroupChat

A team where a generative model (LLM) is used to select the next agent to send a message based on the current conversation history.


In [3]:
llm_team = SelectorGroupChat([tool_use_agent, writing_assistant_agent], model_client=model_client)

llm_team_result = await llm_team.run(
    "What is the weather in paris right now? Also write a haiku about it.",
    termination_condition=MaxMessageTermination(max_messages=2),
)


--------------------------------------------------------------------------- 
[2024-10-20T09:01:05.967894]:

What is the weather in paris right now? Also write a haiku about it.
--------------------------------------------------------------------------- 
[2024-10-20T09:01:07.214716], tool_use_agent:

The weather in Paris is currently 72 degrees and Sunny.

Here's a Haiku about it:

Golden sun above,  
Paris basks in warmth and light,  
Seine flows in sunshine.
--------------------------------------------------------------------------- 
[2024-10-20T09:01:08.320789], writing_assistant_agent:

I can't check the real-time weather, but you can use a weather website or app to find the current weather in Paris. If you need a fresh haiku, here's one for sunny weather:

Paris bathed in sun,  
Gentle warmth embraces all,  
Seine sparkles with light.
--------------------------------------------------------------------------- 
[2024-10-20T09:01:08.321296], Termination:

Maximal number of messages 

## What's Next?

In this section, we reviewed how to define model clients, agents, and teams in AgentChat. Here are some other concepts to explore further:

- Termination Conditions: Define conditions that determine when a team should stop running. In this sample, we used a `MaxMessageTermination` condition to stop the team after a certain number of messages. Explore other termination conditions supported in the AgentChat package.